<a href="https://colab.research.google.com/github/10dimensions/wandb_nn/blob/master/wandb_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [0]:
! git clone https://github.com/lukas/ml-class

Cloning into 'ml-class'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 8971 (delta 36), reused 72 (delta 21), pack-reused 8868
Receiving objects: 100% (8971/8971), 136.37 MiB | 26.61 MiB/s, done.
Resolving deltas: 100% (2131/2131), done.
Checking out files: 100% (5553/5553), done.


In [0]:
%cd /content/ml-class
!ls

/content/ml-class
examples  LICENSE  projects  README.md	requirements.txt  videos


In [0]:
! pip install -r requirements.txt

In [0]:
import wandb

!wandb login 39011b0b79a9067f7ffdab0b8223dc33e75b8801

In [0]:
%cd videos/intro

/content/ml-class/videos/intro


In [0]:
!ls

mlp.py		      perceptron-logistic.py	  perceptron-single.py
mnist.png	      perceptron-normalize.py
perceptron-linear.py  perceptron-single-fixed.py


In [0]:
!sudo apt install emacs

# Single-Layer Perceptron

# Multi-Layer Perceptron

In [0]:
!cat perceptron-linear.py

In [0]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.utils import np_utils

import wandb
from wandb.keras import WandbCallback

# logging code
run = wandb.init()
config = run.config

# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.astype("float")
X_train /= 255.

X_test = X_test.astype("float")
X_test /= 255.

img_width = X_train.shape[1]
img_height = X_train.shape[2]

# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
labels = range(10)

num_classes = y_train.shape[1]

# create model
model=Sequential()
model.add(Flatten(input_shape=(img_width,img_height)))

#additional layer
#model.add(Dense(config.hidden_nodes, activation="relu"))

model.add(Dense(num_classes, activation="softmax"))       #activation = "softmax", improve accuracy
model.compile(loss='categorical_crossentropy', optimizer='adam',    # loss func = mse, categorical cross-entropy
                metrics=['accuracy'])                               # optimizer = config.optimizer

# Fit the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test),
                    callbacks=[WandbCallback(labels=labels, data_type="image")])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 3s 45us/step - loss: 0.4688 - acc: 0.8765 - val_loss: 0.3067 - val_acc: 0.9160
Epoch 2/10
60000/60000 [==============================] - 3s 43us/step - loss: 0.3039 - acc: 0.9154 - val_loss: 0.2902 - val_acc: 0.9194
Epoch 3/10
60000/60000 [==============================] - 3s 44us/step - loss: 0.2833 - acc: 0.9204 - val_loss: 0.2736 - val_acc: 0.9237
Epoch 4/10
60000/60000 [==============================] - 3s 43us/step - loss: 0.2733 - acc: 0.9244 - val_loss: 0.2681 - val_acc: 0.9251
Epoch 5/10
60000/60000 [==============================] - 3s 43us/step - loss: 0.2662 - acc: 0.9261 - val_loss: 0.2678 - val_acc: 0.9260
Epoch 6/10
60000/60000 [==============================] - 3s 43us/step - loss: 0.2615 - acc: 0.9272 - val_loss: 0.2653 - val_acc: 0.9270
Epoch 7/10
60000/60000 [==============================] - 3s 43us/step - loss: 0.2578 - acc: 0.9282 - val_loss: 0.2678 - val_acc

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


# Convolutional Neural Network

In [0]:
%cd videos/cnn/
!ls

/content/ml-class/videos/cnn
cnn.py	conv-demo.py  fashion.py  puppy.jpg


In [0]:
!cat cnn.py

In [0]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from keras.utils import np_utils
from wandb.keras import WandbCallback
import wandb

run = wandb.init(project="ml_class_cnn")
config = run.config
config.img_width = 28
config.img_height = 28
config.first_layer_conv_width = 3
config.first_layer_conv_height = 3
config.dense_layer_size = 100
config.epochs = 10

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.astype('float32')
X_train /= 255.
X_test = X_test.astype('float32')
X_test /= 255.

#reshape input data
X_train = X_train.reshape(X_train.shape[0], config.img_width, config.img_height, 1)
X_test = X_test.reshape(X_test.shape[0], config.img_width, config.img_height, 1)

# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]
labels=range(10)

# build model
model = Sequential()
model.add(Conv2D(32,
    (config.first_layer_conv_width, config.first_layer_conv_height),
    input_shape=(28, 28,1),
    activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(config.dense_layer_size, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam',
                metrics=['accuracy'])

model.summary()

model.fit(X_train, y_train, validation_data=(X_test, y_test),
        epochs=config.epochs,
        callbacks=[WandbCallback(data_type="image")])

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


11493376/11490434 [==============================] - 0s 0us/step





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5408)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               540900    
_________________________________________________________________

# Auto-encoder

In [0]:
%cd /content/ml-class/videos/autoencoder/
!ls

/content/ml-class/videos/autoencoder
autoencoder_cnn.py  denoising_autoencoder.py
autoencoder.py	    run_autoencoder.py


In [0]:
!cat autoencoder.py

In [0]:
from keras.layers import Input, Dense, Flatten, Reshape
from keras.models import Model, Sequential

from keras.datasets import mnist
from keras.callbacks import Callback
import numpy as np
import wandb
from wandb.keras import WandbCallback

run = wandb.init(project="ml_class_autoenc")
config = run.config

config.encoding_dim = 32
config.epochs = 10

(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

model = Sequential()
model.add(Flatten(input_shape=(28,28)))
model.add(Dense(config.encoding_dim, activation='relu'))
model.add(Dense(28*28, activation='sigmoid'))
model.add(Reshape((28,28)))
model.compile(optimizer='adam', loss='mse')

# For visualization
class Images(Callback):
      def on_epoch_end(self, epoch, logs):
            indices = np.random.randint(self.validation_data[0].shape[0], size=8)
            test_data = self.validation_data[0][indices]
            pred_data = self.model.predict(test_data)
            run.history.row.update({
                  "examples": [
                        wandb.Image(np.hstack([data, pred_data[i]]), caption=str(i))
                        for i, data in enumerate(test_data)]
            })

model.summary()
model.fit(x_train, x_train,
                epochs=config.epochs,
                validation_data=(x_test, x_test), 
          callbacks=[Images(), WandbCallback()])


model.save('auto.h5')

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                25120     
_________________________________________________________________
dense_4 (Dense)              (None, 784)               25872     
_________________________________________________________________
reshape_2 (Reshape)          (None, 28, 28)            0         
Total params: 50,992
Trainable params: 50,992
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 8s 135us/step - loss: 0.0325 - val_loss: 0.0162
Epoch 2/10
60000/60000 [==============================] - 8s 131us/step - loss: 0.0134 - val

# Sentiment Analysis

In [0]:
cd videos/text-classifier/

/content/ml-class/videos/text-classifier


In [0]:
more tweets.csv

In [0]:
# Quick example of loading our data into variables

import pandas as pd
import numpy as np

# Puts tweets into a data frame
df = pd.read_csv('tweets.csv')

# Selects the first column from our data frame
target = df['is_there_an_emotion_directed_at_a_brand_or_product']

# Selects the third column from our data frame
text = df['tweet_text']

print(len(text))


9093


**Feature Extraction**

In [0]:
# First attempt at feature extraction
# Leads to an error, can you tell why?

import pandas as pd
import numpy as np

df = pd.read_csv('tweets.csv')
target = df['is_there_an_emotion_directed_at_a_brand_or_product']
text = df['tweet_text']

# what did we do here?
fixed_text = text[pd.notnull(text)]
fixed_target = target[pd.notnull(text)]

from sklearn.feature_extraction.text import CountVectorizer

count_vect=CountVectorizer()
count_vect.fit(fixed_text)

# turns the text into a sparse matrix
counts = count_vect.transform(fixed_text)

my_counts = count_vect.transform(["love that iphone!", "HATE that iphone"])
print(my_counts)

  (0, 4573)	1
  (0, 5169)	1
  (0, 8560)	1
  (1, 3975)	1
  (1, 4573)	1
  (1, 8560)	1


**Classifier**

In [0]:
import pandas as pd
import numpy as np

# Get a pandas DataFrame object of all the data in the csv file:
df = pd.read_csv('tweets.csv')

# Get pandas Series object of the "tweet text" column:
text = df['tweet_text']

# Get pandas Series object of the "emotion" column:
target = df['is_there_an_emotion_directed_at_a_brand_or_product']

# Remove the blank rows from the series:
target = target[pd.notnull(text)]
text = text[pd.notnull(text)]

# Perform feature extraction:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
count_vect.fit(text)
counts = count_vect.transform(text)

# Train with this data with a Naive Bayes classifier:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(counts, target)

#Try the classifier
print(nb.predict(count_vect.transform(['i hate my iphone'])))


['Negative emotion']


In [0]:
print(nb.predict(count_vect.transform(['i love my iphone'])))

['Positive emotion']


# RNN/Time-Series

In [0]:
cd /content/ml-class/videos/time-series/

In [0]:
!wandb login 39011b0b79a9067f7ffdab0b8223dc33e75b8801

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [0]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import LSTM, SimpleRNN, Dropout
from keras.callbacks import LambdaCallback

import wandb
from wandb.keras import WandbCallback

import plotutil
from plotutil import PlotCallback

wandb.init("ml_class_rnn")
config = wandb.config

config.repeated_predictions = False
config.look_back = 20

def load_data(data_type="airline"):
    if data_type == "flu":
        df = pd.read_csv('flusearches.csv')
        data = df.flu.astype('float32').values
    elif data_type == "airline":
        df = pd.read_csv('international-airline-passengers.csv')
        data = df.passengers.astype('float32').values
    elif data_type == "sin":
        df = pd.read_csv('sin.csv')
        data = df.sin.astype('float32').values
    return data

# convert an array of values into a dataset matrix
def create_dataset(dataset):
    dataX, dataY = [], []
    for i in range(len(dataset)-config.look_back-1):
        a = dataset[i:(i+config.look_back)]
        dataX.append(a)
        dataY.append(dataset[i + config.look_back])
    return np.array(dataX), np.array(dataY)

data = load_data()
    
# normalize data to between 0 and 1
max_val = max(data)
min_val = min(data)
data=(data-min_val)/(max_val-min_val)

# split into train and test sets
split = int(len(data) * 0.70)
train = data[:split]
test = data[split:]

trainX, trainY = create_dataset(train)
testX, testY = create_dataset(test)

trainX = trainX[:, :, np.newaxis]
testX = testX[:, :, np.newaxis]

# create and fit the RNN
model = Sequential()
model.add(Flatten(input_shape=(config.look_back,1 )))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
model.fit(trainX, trainY, epochs=1000, batch_size=10, validation_data=(testX, testY),  callbacks=[WandbCallback(), PlotCallback(trainX, trainY, testX, testY, config.look_back)])







Train on 79 samples, validate on 23 samples
Epoch 1/1000
79/79 [==============================] - 0s 3ms/step - loss: 0.0656 - val_loss: 0.3502
Epoch 2/1000
79/79 [==============================] - 0s 518us/step - loss: 0.0511 - val_loss: 0.2641
Epoch 3/1000
79/79 [==============================] - 0s 598us/step - loss: 0.0379 - val_loss: 0.2076
Epoch 4/1000
79/79 [==============================] - 0s 554us/step - loss: 0.0310 - val_loss: 0.1669
Epoch 5/1000
79/79 [==============================] - 0s 597us/step - loss: 0.0263 - val_loss: 0.1421
Epoch 6/1000
79/79 [==============================] - 0s 448us/step - loss: 0.0239 - val_loss: 0.1278
Epoch 7/1000
79/79 [==============================] - 0s 790us/step - loss: 0.0228 - val_loss: 0.1195
Epoch 8/1000
79/79 [==============================] - 0s 543us/step - loss: 0.0222 - val_loss: 0.1154
Epoch 9/1000
79/79 [==============================] - 0s 568us/step - loss: 0.0219 - val_loss: 0.1131
Epoch 10/1000
79/79 [===================

**RNN**

In [0]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import LSTM, SimpleRNN, Dropout
from keras.callbacks import LambdaCallback

import wandb
from wandb.keras import WandbCallback

import plotutil
from plotutil import PlotCallback

wandb.init("ml_class_rnn")
config = wandb.config

config.repeated_predictions = False
config.look_back = 20

df = pd.read_csv('international-airline-passengers.csv')
data = df.passengers.astype('float32').values

# convert an array of values into a dataset matrix
def create_dataset(dataset):
    dataX, dataY = [], []
    for i in range(len(dataset)-config.look_back-1):
        a = dataset[i:(i+config.look_back)]
        dataX.append(a)
        dataY.append(dataset[i + config.look_back])
    return np.array(dataX), np.array(dataY)

data = load_data()
    
# normalize data to between 0 and 1
max_val = max(data)
min_val = min(data)
data=(data-min_val)/(max_val-min_val)

# split into train and test sets
split = int(len(data) * 0.70)
train = data[:split]
test = data[split:]

trainX, trainY = create_dataset(train)
testX, testY = create_dataset(test)

trainX = trainX[:, :, np.newaxis]
testX = testX[:, :, np.newaxis]

# create and fit the RNN
model = Sequential()
model.add(SimpleRNN(5, input_shape=(config.look_back,1 )))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='mae', optimizer='adam')
model.fit(trainX, trainY, epochs=1000, batch_size=1, validation_data=(testX, testY),  callbacks=[WandbCallback(), PlotCallback(trainX, trainY, testX, testY, config.look_back)])



Train on 79 samples, validate on 23 samples
Epoch 1/1000
79/79 [==============================] - 2s 31ms/step - loss: 0.3091 - val_loss: 0.1104
Epoch 2/1000
79/79 [==============================] - 2s 24ms/step - loss: 0.1071 - val_loss: 0.2403
Epoch 3/1000
79/79 [==============================] - 2s 24ms/step - loss: 0.0814 - val_loss: 0.2045
Epoch 4/1000
79/79 [==============================] - 2s 23ms/step - loss: 0.0724 - val_loss: 0.1075
Epoch 5/1000
79/79 [==============================] - 2s 24ms/step - loss: 0.0655 - val_loss: 0.1037
Epoch 6/1000
79/79 [==============================] - 2s 22ms/step - loss: 0.0605 - val_loss: 0.1039
Epoch 7/1000
79/79 [==============================] - 2s 23ms/step - loss: 0.0597 - val_loss: 0.1046
Epoch 8/1000
79/79 [==============================] - 2s 24ms/step - loss: 0.0575 - val_loss: 0.1022
Epoch 9/1000
79/79 [==============================] - 2s 24ms/step - loss: 0.0552 - val_loss: 0.1077
Epoch 10/1000
79/79 [==========================

# LSTM

In [0]:
cd /content/ml-class/videos/text-gen

/content/ml-class/videos/text-gen


In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM, SimpleRNN, GRU
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io
import wandb
from wandb.keras import WandbCallback
import argparse

parser = argparse.ArgumentParser()
parser.add_argument("text", type=str)

args = parser.parse_args()

run = wandb.init()
config = run.config
config.hidden_nodes = 128
config.batch_size = 256
config.file = args.text
config.maxlen = 200
config.step = 3

text = io.open(config.file, encoding='utf-8').read()
chars = sorted(list(set(text)))

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# build a sequence for every <config.step>-th character in the text

sentences = []
next_chars = []
for i in range(0, len(text) - config.maxlen, config.step):
    sentences.append(text[i: i + config.maxlen])
    next_chars.append(text[i + config.maxlen])

# build up one-hot encoded input x and output y where x is a character
# in the text y is the next character in the text

x = np.zeros((len(sentences), config.maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

model = Sequential()
model.add(SimpleRNN(128, input_shape=(config.maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer="rmsprop")


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

class SampleText(keras.callbacks.Callback):
    def on_epoch_end(self, batch, logs={}):
        start_index = random.randint(0, len(text) - config.maxlen - 1)

        for diversity in [0.5, 1.2]:
            print()
            print('----- diversity:', diversity)

            generated = ''
            sentence = text[start_index: start_index + config.maxlen]
            generated += sentence
            print('----- Generating with seed: "' + sentence + '"')
            sys.stdout.write(generated)

            for i in range(200):
                x_pred = np.zeros((1, config.maxlen, len(chars)))
                for t, char in enumerate(sentence):
                    x_pred[0, t, char_indices[char]] = 1.

                preds = model.predict(x_pred, verbose=0)[0]
                next_index = sample(preds, diversity)
                next_char = indices_char[next_index]

                generated += next_char
                sentence = sentence[1:] + next_char

                sys.stdout.write(next_char)
                sys.stdout.flush()
            print()
            
model.fit(x, y, batch_size=config.batch_size,
              epochs=100, callbacks=[SampleText(), WandbCallback()])


In [0]:
! python char-gen.py

Using TensorFlow backend.
usage: char-gen.py [-h] text
char-gen.py: error: the following arguments are required: text
